# K-Nearest

Sirve esencialmente para clasificar valores buscando los puntos de datos “más similares” (por cercanía) aprendidos en la etapa de entrenamiento.
Clasifica cada punto en una categoría, basándose en la categoría de sus vecinos más cercanos. 
Suele utilizarse en sistemas de recomendación, búsqueda semántica y detección de anomalías. 

    https://www.aprendemachinelearning.com/clasificar-con-k-nearest-neighbor-ejemplo-en-python/#:~:text=K%2DNearest%2DNeighbor%20es%20un,el%20mundo%20del%20Aprendizaje%20Autom%C3%A1tico.
    
    Pros: Sencillo de aprender e implementar. Robusto y versátil.
    Contras: Utiliza todo el dataset para entrenar “cada punto” y por eso requiere de uso de mucha memoria y recursos CPU. 
    
    Cómo funciona:
        1. Calcula la distancia entre el item a clasificar y el resto de items del dataset de entrenamiento.
        2. Selecciona los “k” elementos más cercanos (con menor distancia, según la función que se use)
        3. Realiza una “votación de mayoría” entre los k puntos: los de una clase/etiqueta que <<dominen>> decidirán su clasificación final.

## K-Nearest profe

In [2]:
# Clasifica cada punto en una categoría dependiendo basándose en la categoría de sus vecinos más cercanos.
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

df = pd.read_csv("base_datos_2008.csv", nrows = 1e6)

In [3]:
newdf = df[["AirTime", "Distance", "TaxiOut", "ArrDelay"]].dropna()
cols = newdf[newdf.columns[newdf.columns != "ArrDelay"]]

In [4]:
filtro = newdf["ArrDelay"] > 10

In [5]:
# Ponemos etiquetas a nuestros casos.
newdf["ArrDelay"][filtro] = "Delayed"
newdf["ArrDelay"][filtro == False] = "Not Delayed"

In [6]:
newdf["ArrDelay"].head()

0    Not Delayed
1    Not Delayed
2        Delayed
3    Not Delayed
4        Delayed
Name: ArrDelay, dtype: object

In [7]:
# Sugiero utilizar un número impar para la comparación entre vecinos, de forma que podamos dilucidar posibles empates...

nbrs_3 = KNeighborsClassifier(n_neighbors = 3, n_jobs = -1)

In [8]:
nbrs_3.fit(cols, newdf["ArrDelay"])

KNeighborsClassifier(n_jobs=-1, n_neighbors=3)

In [9]:
predicciones_3 = nbrs_3.predict(cols)

In [10]:
np.mean(predicciones_3 == newdf["ArrDelay"])

0.8040059625362947

In [11]:
# El 80,4% están clasificados correctamente, esta medida puede ser engañosa, ya que no conocemos la proporción de nuestros datos
np.mean(newdf["ArrDelay"] == "Not Delayed")

0.6912450368307814

In [12]:
# Casi el 70% de vuelos no se han retrasado, con el modelo estamos indicando 80,4%, hay una desviación de 10 puntos.
# Vamos a ir ajustando mejor la métrica, miramos solo el vecino de al lado.
nbrs_1 = KNeighborsClassifier(n_neighbors = 1, n_jobs = -1)
nbrs_1.fit(cols, newdf["ArrDelay"])
predicciones_1 = nbrs_1.predict(cols)
np.mean(predicciones_1 == newdf["ArrDelay"])

0.8346359548396921

In [13]:
# Observando el vecino más cercano obtenemos una mejor predicción que observando los tres más cercanos.
np.mean(newdf["ArrDelay"] == "Not Delayed")

0.6912450368307814

In [14]:
# Exploramos la matriz de confusión para ver cuánto de bien hemos predicho los casos.
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(newdf["ArrDelay"], predicciones_1)
print(confusion_matrix)

[[218051  80009]
 [ 79627 587674]]


In [15]:
# (0,0) -> En este caso el valor real de los datos eran 0 (False = con retraso) y el modelo ha predicho como 0 (con retraso).
# (0,1) -> En este caso el valor real de los datos eran 0 (False = con retraso) y el modelo ha predicho como 1 (sin retraso).
# (1,0) -> En este caso el valor real de los datos eran 1 (True = sin retraso) y el modelo ha predicho como 0 (con retraso).
# (1,1) -> En este caso el valor real de los datos eran 1 (True = sin retraso) y el modelo ha predicho como 1 (sin retraso).

## K-Nearest Pokemon

In [20]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

dfP = pd.read_csv("pokemon.csv")

Podemos utilizar este algoritmo para ver cuáles son los Pokémon más fuertes, tratando la suma de estadísticas (base_total):

In [21]:
dfP = dfP[["base_total"]].dropna()

In [22]:
dfP.describe()

,base_total
count,801.000000
mean,428.377029
std,119.203577
min,180.000000
25%,320.000000
50%,435.000000
75%,505.000000
max,780.000000


Vemos cuántos Pokémon tienen un sumatorio de 500 puntos de estadísticas, divididos en Ataque, At.Esp., Defensa, Def.Esp. y Velocidad

In [23]:
filtro = dfP["base_total"] > 500

In [24]:
# Ponemos etiquetas
dfP["base_total"][filtro] = "Fuerte"
dfP["base_total"][filtro == False] = "Normal-Bajo"

In [25]:
dfP["base_total"].head()

0    Normal-Bajo
1    Normal-Bajo
2         Fuerte
3    Normal-Bajo
4    Normal-Bajo
Name: base_total, dtype: object

In [26]:
# Sugiero utilizar un número impar para la comparación entre vecinos, de forma que podamos dilucidar posibles empates...

nbrs_3 = KNeighborsClassifier(n_neighbors = 3, n_jobs = -1)

In [27]:
nbrs_3.fit(cols, dfP["base_total"])

ValueError: Found input variables with inconsistent numbers of samples: [965361, 801]